<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Nand" data-toc-modified-id="Nand-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Nand</a></span></li><li><span><a href="#Not-(reverse-bit)" data-toc-modified-id="Not-(reverse-bit)-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Not (reverse bit)</a></span></li><li><span><a href="#Swap-columns" data-toc-modified-id="Swap-columns-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Swap columns</a></span></li><li><span><a href="#And-(restrict-bits)" data-toc-modified-id="And-(restrict-bits)-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>And (restrict bits)</a></span><ul class="toc-item"><li><span><a href="#Demonstration" data-toc-modified-id="Demonstration-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Demonstration</a></span></li></ul></li><li><span><a href="#Or-(union-bits)" data-toc-modified-id="Or-(union-bits)-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Or (union bits)</a></span><ul class="toc-item"><li><span><a href="#Demonstration" data-toc-modified-id="Demonstration-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Demonstration</a></span></li></ul></li><li><span><a href="#Nor" data-toc-modified-id="Nor-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Nor</a></span><ul class="toc-item"><li><span><a href="#Demonstration" data-toc-modified-id="Demonstration-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Demonstration</a></span></li></ul></li><li><span><a href="#Operators" data-toc-modified-id="Operators-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Operators</a></span></li><li><span><a href="#Always-on" data-toc-modified-id="Always-on-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Always on</a></span><ul class="toc-item"><li><span><a href="#Demonstration" data-toc-modified-id="Demonstration-8.1"><span class="toc-item-num">8.1&nbsp;&nbsp;</span>Demonstration</a></span></li></ul></li><li><span><a href="#Second-tick" data-toc-modified-id="Second-tick-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Second tick</a></span><ul class="toc-item"><li><span><a href="#Demonstration" data-toc-modified-id="Demonstration-9.1"><span class="toc-item-num">9.1&nbsp;&nbsp;</span>Demonstration</a></span></li></ul></li><li><span><a href="#Xor" data-toc-modified-id="Xor-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>Xor</a></span><ul class="toc-item"><li><span><a href="#Demonstration" data-toc-modified-id="Demonstration-10.1"><span class="toc-item-num">10.1&nbsp;&nbsp;</span>Demonstration</a></span></li></ul></li></ul></div>

In [1]:
# !pip install --upgrade pip setuptools wheel
# !pip install pyeda
# !pip install schemdraw

In [2]:
from itertools import product
import pandas as pd
from pyeda.inter import *

a, b, c = map(exprvar, 'abc')


def truth_table_to_pandas(table):
    df = pd.DataFrame()
    for variables, result in table.iter_relation():
        variables["result"] = result
        variables = {k: [v] for (k, v) in variables.items()}
        df = pd.concat([df, pd.DataFrame(variables)], ignore_index=True)
    return df


def dict_to_truthtable(d):
    inputs = list(d.keys())[0]
    n_vars = len(inputs)
    variables = exprvars('', n_vars)
    output_vals = [d[tuple(inputs)] for inputs in product(*[[0, 1] for _ in range(n_vars)])]
    return truthtable(variables, output_vals)


def is_truth_tables_equivalent(*tables):
    cursor = truth_table_to_pandas(tables[0])["result"]
    for table in tables:
        if type(table) is TruthTable:
            table = truth_table_to_pandas(table)["result"]
        else:
            table = truth_table_to_pandas(expr2truthtable(table))["result"]
        if not table.equals(cursor):
            return False
    return True

def p(obj):
    print(obj)
    
def pt(expr):
    table = expr2truthtable(expr)
    # print(expr2truthtable(expr))
    print(truth_table_to_pandas(expr).T.to_string(header=False))
    print()

## Nand

In [3]:
NAND_1 = Nand(a, b)
NAND_2 = ~(a & b)
pt(NAND_1)
Nand(a, b).equivalent(~(a & b))

a       0  1  0  1
b       0  0  1  1
result  1  1  1  0



True

In [4]:
NAND_1.restrict({a:1, b:0})

1

## Not (reverse bit)

In [5]:
NOT_1, NOT_2 = Not(a), ~a
pt(NOT_1); pt(~a)
Not(a).equivalent(~a)

a       0  1
result  1  0

a       0  1
result  1  0



True

## Swap columns

In [6]:
table = truth_table_to_pandas(And(a, b))
table["And(~a, b)"] = truth_table_to_pandas(And(~a, b))["result"]
table["And(a, ~b)"] = truth_table_to_pandas(And(a, ~b))["result"]
table["And(~a, ~b)"] = truth_table_to_pandas(And(~a, ~b))["result"]
table.T

,0,1,2,3
a,0,1,0,1
b,0,0,1,1
result,0,0,0,1
"And(~a, b)",0,0,1,0
"And(a, ~b)",0,1,0,0
"And(~a, ~b)",1,0,0,0


In [7]:
table = truth_table_to_pandas(Xor(a, b))
table["Xor(~a, b)"] = truth_table_to_pandas(Xor(~a, b))["result"]
table["Xor(a, ~b)"] = truth_table_to_pandas(Xor(a, ~b))["result"]
table["Xor(~a, ~b)"] = truth_table_to_pandas(Xor(~a, ~b))["result"]
table.T

,0,1,2,3
a,0,1,0,1
b,0,0,1,1
result,0,1,1,0
"Xor(~a, b)",1,0,0,1
"Xor(a, ~b)",1,0,0,1
"Xor(~a, ~b)",0,1,1,0


## And (restrict bits)

In [8]:
AND_1 = And(a, b)
AND_2 = a & b
pt(AND_1)
And(a, b).equivalent(a & b)

a       0  1  0  1
b       0  0  1  1
result  0  0  0  1



True

### Demonstration

In [9]:
r = ~(a & b); pt(r)
r = ~~(a & b); pt(r)
(~~(a & b)).equivalent(And(a, b))

a       0  1  0  1
b       0  0  1  1
result  1  1  1  0

a       0  1  0  1
b       0  0  1  1
result  0  0  0  1



True

## Or (union bits)

In [10]:
OR_1 = Or(a, b)
OR_2 = a | b
pt(OR_1)
Or(a, b).equivalent(a | b)

a       0  1  0  1
b       0  0  1  1
result  0  1  1  1



True

### Demonstration

In [11]:
r = a & b; pt(r)
r = ~a & ~b; pt(r)
r = Nand(~a, ~b); pt(r)
Nand(~a & ~b).equivalent(Or(a, b)), (~(~a & ~b))

a       0  1  0  1
b       0  0  1  1
result  0  0  0  1

a       0  1  0  1
b       0  0  1  1
result  1  0  0  0

a       0  1  0  1
b       0  0  1  1
result  0  1  1  1



(True, Not(And(~a, ~b)))

## Nor

In [12]:
NOR_1 = Nor(a, b)
NOR_1 = ~(a | b)
pt(OR_1)
Nor(a, b).equivalent(~(a | b))

a       0  1  0  1
b       0  0  1  1
result  0  1  1  1



True

### Demonstration

In [13]:
r = Or(a, b); pt(r)
r = ~(a | b); pt(r)
(~(a | b)).equivalent(Nor(a, b))

a       0  1  0  1
b       0  0  1  1
result  0  1  1  1

a       0  1  0  1
b       0  0  1  1
result  1  0  0  0



True

## Operators

In [14]:
~a, a & b, a | b, a ^ b, a | b, a >> b

(~a, And(a, b), Or(a, b), Xor(a, b), Or(a, b), Implies(a, b))

## Always on

In [15]:
always_on = dict_to_truthtable({(0,): 1,
                                (1,): 1})
pt(always_on)

[0]     0  1
result  1  1



### Demonstration

In [16]:
r = ~a; pt(r)
r = ~a | a; pt(r)
r = ~(~a & a); pt(r)
is_truth_tables_equivalent(always_on, ~(~a & a))

a       0  1
result  1  0

a       0  1
result  1  1

a       0  1
result  1  1



True

## Second tick

In [17]:
second_tick = dict_to_truthtable({(0, 0): 0,
                                  (0, 1): 1,
                                  (1, 0): 0,
                                  (1, 1): 0})
pt(second_tick)

[0]     0  1  0  1
[1]     0  0  1  1
result  0  1  0  0



### Demonstration

In [18]:
r = a & b; pt(r)
r = a & ~b; pt(r)
r = ~(~a | b); pt(r)
is_truth_tables_equivalent(second_tick, a & ~b)

a       0  1  0  1
b       0  0  1  1
result  0  0  0  1

a       0  1  0  1
b       0  0  1  1
result  0  1  0  0

a       0  1  0  1
b       0  0  1  1
result  0  1  0  0



True

## Xor

In [19]:
XOR_1 = Xor(a, b)
XOR_2 = a ^ b
pt(XOR_1)
Xor(a, b).equivalent(a ^ b)

a       0  1  0  1
b       0  0  1  1
result  0  1  1  0



True

### Demonstration

In [20]:
r = ~(a & b); pt(r)
r = (a | b); pt(r)
r = ~(a & b) & (a | b); pt(r)

a       0  1  0  1
b       0  0  1  1
result  1  1  1  0

a       0  1  0  1
b       0  0  1  1
result  0  1  1  1

a       0  1  0  1
b       0  0  1  1
result  0  1  1  0



In [21]:
r = (a | b) | c; pt(r)

a       0  1  0  1  0  1  0  1
b       0  0  1  1  0  0  1  1
c       0  0  0  0  1  1  1  1
result  0  1  1  1  1  1  1  1

